In [2]:
import requests
import pandas as pd
import time

In [ ]:
def fetch_toplist_page(page_num):
    url = (
        "https://worldathletics.org/records/all-time-toplists/jumps/high-jump/all/women/u18"
        "?regionType=world"
        "&bestResultsOnly=false"
        "&firstDay=1899-12-31"
        "&lastDay=2025-10-19"
        "&maxResultsByCountry=all"
        "&eventId=10229526"
        "&ageCategory=u18"
        f"&page={page_num}"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text

In [ ]:
def parse_table_from_html(html):
    dfs = pd.read_html(html)
    for df in dfs:
        cols = set(df.columns)
        expected = {"Rank", "Mark", "WIND", "Competitor", "DOB", "Country", "Pos", "Venue", "Date", "Results Score"}
        if expected.intersection(cols):
            return df
        return max(dfs, key=lambda d: d.shape[0])

In [6]:
def scrape_all_pages(max_pages=40, delay=1.0):
    all_dfs = []
    for page in range(1, max_pages + 1):
        print(f"Fetching page {page}...")
        html = fetch_toplist_page(page)
        df = parse_table_from_html(html)
        if df is None or df.shape[0] == 0:
            print("No more data or empty table on this page. Stopping.")
            break
        all_dfs.append(df)
        time.sleep(delay)
    combined = pd.concat(all_dfs, ignore_index=True)
    return combined

In [8]:
def main():
    df = scrape_all_pages(max_pages=10, delay=1.0)
    print("Scraped rows:", len(df))
    df.to_csv("women_u18_high_jump.csv", index=False)
    print("Saved CSV.")

if __name__ == "__main__":
    main()

Fetching page 1...


NameError: name 'request' is not defined